In [2]:
from pikepdf import Pdf, PdfImage
import pytesseract

from PIL import Image
import re
import cv2
import numpy as np
from matplotlib import pyplot as plt

from pyzbar.pyzbar import decode

In [6]:
with Pdf.open('1.pdf') as pdf:

    page = pdf.pages[0]
    keyimage = list(page.images.keys())
    rawimage = page.images[keyimage[0]]
    pdfimage = PdfImage(rawimage)
    image = pdfimage.as_pil_image()

if pdfimage.width / pdfimage.height < 0.6:
    rotation_degrees = re.findall("\d+", pytesseract.image_to_osd(image).split('\n')[1])[0]
    if rotation_degrees != '0':
        image = image.rotate(int(rotation_degrees) , expand=True)

In [8]:


cv2_img = np.array(image)

# 灰度图片
gray = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2GRAY)
# 二值化
binary = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 35, -5)
cv2.imshow("binary_picture", binary)  # 展示图片
rows, cols = binary.shape
scale = 40
# 自适应获取核值 识别横线
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (cols // scale, 1))
eroded = cv2.erode(binary, kernel, iterations=1)

dilated_col = cv2.dilate(eroded, kernel, iterations=1)
cv2.imshow("excel_horizontal_line", dilated_col)
# cv2.waitKey(0)
# 识别竖线
scale = 20
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, rows // scale))
eroded = cv2.erode(binary, kernel, iterations=1)
dilated_row = cv2.dilate(eroded, kernel, iterations=1)
cv2.imshow("excel_vertical_line", dilated_row)
# cv2.waitKey(0)


# 标识交点
bitwise_and = cv2.bitwise_and(dilated_col, dilated_row)
cv2.imshow("excel_bitwise_and", bitwise_and)
# cv2.waitKey(0)


# 标识表格
merge = cv2.add(dilated_col, dilated_row)
cv2.imshow("entire_excel_contour", merge)
# cv2.waitKey(0)

# 
# 识别黑白图中的白色交叉点，将横纵坐标取出
ys, xs = np.where(bitwise_and > 0)

# print(ys, xs)
# 纵坐标
y_point_arr = []
# 横坐标
x_point_arr = []
# 通过排序，获取跳变的x和y的值，说明是交点，否则交点会有好多像素值值相近，我只取相近值的最后一点
# 这个10的跳变不是固定的，根据不同的图片会有微调，基本上为单元格表格的高度（y坐标跳变）和长度（x坐标跳变）
i = 0
sort_x_point = np.sort(xs)
for i in range(len(sort_x_point) - 1):
    if sort_x_point[i + 1] - sort_x_point[i] > 20:
        x_point_arr.append(sort_x_point[i])
    i = i + 1
x_point_arr.append(sort_x_point[i])  # 要将最后一个点加入

i = 0
sort_y_point = np.sort(ys)
# print(np.sort(ys))
for i in range(len(sort_y_point) - 1):
    if (sort_y_point[i + 1] - sort_y_point[i] > 20):
        y_point_arr.append(sort_y_point[i])
    i = i + 1
# 要将最后一个点加入
y_point_arr.append(sort_y_point[i])
print('y_point_arr', y_point_arr)
print('x_point_arr', x_point_arr)


# 来个范围筛选
for i in range(len(x_point_arr)):
    if x_point_arr[i] > 100:
        break

for j in range(len(y_point_arr)):
    if y_point_arr[j] > 300:
        j = j - 1
        break
    elif y_point_arr[j] > 100:
        break

left, top, right, bottom = x_point_arr[i+1], y_point_arr[j+1], x_point_arr[i+6], y_point_arr[j+2]
# cv2.rectangle(raw, (x_point_arr[i+1], y_point_arr[1]), (x_point_arr[i+4], y_point_arr[2]), (0, 0, 255), 2)

y_point_arr [143, 238, 409, 753, 819, 976]
x_point_arr [157, 219, 573, 777, 848, 874, 1035, 1075, 1112, 1191, 1428, 1511, 1744]


In [9]:

img = image.crop((left, top, right, bottom))
# img.show()
text = pytesseract.image_to_string(img, lang='eng+chi_sim')
print(text)

名 称 : 波 科 国际 医疗 贸易 《上 海 ) 有 限 公司

纳税 人 识别 号 : 91310000607379141

地 址 、 电 话 : HR CE) 自由 贸易 试验 区 日 京 路 68 号 生产 楼 第 二 层 A 部 位 021-
61415959

开户 行 及 账号 : 清 打 银行 CHIL) 有 限 公司 上 海 分 行 404061-4037580133




In [12]:
reg_tax = r'91[0-9A-Za-z]*'
# reg_tax2 = r'[纳税人识别号].*?(\d+)'
reg_com = r'波科|国际|医疗'
tax_real = '913100006073791417'

tax_reco_s = re.findall(reg_tax, text)
com_reco_s = re.findall(reg_com, text)


com_reco_s

['国际', '医疗']

: 

In [ ]:
# 输出二维码坐标

from PIL import Image, ImageDraw

with Pdf.open('22.pdf') as pdf:

    page = pdf.pages[0]
    keyimage = list(page.images.keys())
    rawimage = page.images[keyimage[0]]
    pdfimage = PdfImage(rawimage)
    img = pdfimage.as_pil_image()


img1 = img.crop((0, 0, img.size[0] / 4, img.size[1] / 4))

barcodes = decode(img1)

left_top, _, botton_right, _ = barcodes[0].polygon

print(left_top.x, left_top.y, botton_right.x, botton_right.y)

In [ ]:
with Pdf.open('1.pdf') as pdf:

    page = pdf.pages[0]
    keyimage = list(page.images.keys())
    rawimage = page.images[keyimage[0]]
    pdfimage = PdfImage(rawimage)
    image = pdfimage.as_pil_image()

if pdfimage.width / pdfimage.height < 0.6:
    rotation_degrees = re.findall("\d+", pytesseract.image_to_osd(image).split('\n')[1])[0]
    if rotation_degrees != '0':
        image = image.rotate(int(rotation_degrees) , expand=True)

text = pytesseract.image_to_string(image, lang='eng+chi_sim')
print(text)

In [52]:
img = cv2.imread('1.png')
# 灰度处理
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

edges = cv2.Canny(gray,50,120)

text = pytesseract.image_to_string(gray)
print(text)
# lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, lines=100, minLineLength=20)

# # for x1,y1,x2,y2 in lines:
# #     cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

# img1 = Image.fromarray(edges)
# img1.show()
# 91510100MA6CWUW177


~91510100MA6CWUW177



In [3]:
img= cv2.imread('22.png')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
decode(imgray)

[]